In [1]:
# -*- coding: utf-8 -*-
##############
#  Packages  #
##############
import os
import sys 


from pathlib import Path
import seaborn as sns 
import plotly.express as px
from plotly.offline import plot
import plotly.io as pio

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#from umap import UMAP

from tqdm import tqdm

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import matplotlib.pyplot as plt


pio.renderers = "browser"

##################
#      Imports   #
##################
root_path = Path("C:/Users/Charles/Desktop/MVA/GDA/Geom_stat/")
sys.path.insert(0, str(root_path))

from utils_tda_and_clustering import (
    homology_parquet_to_matrix_bootstraps,
    make_pca_bootstraps,
    get_clusters
)

img_path = root_path.joinpath("raw_images")
saving_path = root_path.joinpath("outputs")

In [2]:
base_name = "b1"
base_path = str(saving_path.joinpath(f"{base_name}.parquet"))
df_ident, bootstraps, original = homology_parquet_to_matrix_bootstraps(base_path)

100%|██████████| 100/100 [06:43<00:00,  4.04s/it]


In [3]:
reduced_bootstraps, reduced_original, vars_explained = make_pca_bootstraps(bootstraps, original)

In [4]:
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist

def optimalK(reduced_bootstraps,  reduced_original, minClusters=2, maxClusters=10):
    """
    Calculates KMeans optimal K using Gap Statistic 
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(minClusters, maxClusters)),))
    resultsdf = {}
    for gap_index, k in enumerate(range(minClusters, maxClusters)):# Holder for reference dispersion results
        refDisps = np.zeros(len(bootstraps))# For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i,b in tqdm(enumerate(reduced_bootstraps)):
            clustering_model = AgglomerativeClustering(n_clusters=k, metric='euclidean', linkage='ward')
            # linked = linkage(b, "ward")
            clustering_model.fit(b)
            clusters_indices = clustering_model.labels_
            # clusters_indices = fcluster(linked, k, criterion="maxclust")
            clusters = [np.array([b[j] for j in range(len(clusters_indices)) if clusters_indices[j]==i]) for i in range(k)]
            
            distances = []
            for c in clusters:
                D_c = np.sum(pdist(c, 'euclidean')**2)/(2*len(c))
                distances.append(D_c)
            
            bootstrapDisp = np.sum(distances) # The value of W_k for one of our bootstraps
            refDisps[i] = bootstrapDisp
        
        # linked = linkage(reduced_original, "ward")
        # clusters_indices = fcluster(linked, k, criterion="maxclust")
        clustering_model = AgglomerativeClustering(n_clusters=k, metric='euclidean', linkage='ward')
        clustering_model.fit(reduced_original)
        clusters_indices = clustering_model.labels_
        
        clusters = [[b[j] for j in range(len(clusters_indices)) if clusters_indices[j]==i] for i in range(k)]
        
        distances = []
        for c in clusters:
            D_c = np.sum(pdist(c, 'euclidean')**2)/(2*len(c))
            distances.append(D_c)
        
        origDisp = np.sum(distances)
        
        gap = np.mean(np.log(refDisps)) - np.log(origDisp)# Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        resultsdf[k] = gap
    return (gaps.argmax() + minClusters, resultsdf)

In [5]:
from utils_tda_and_clustering import get_clusters

In [6]:
clustered_bootstraps, clustered_original = get_clusters(reduced_bootstraps, reduced_original)

In [7]:
from utils_tda_and_clustering import transform_gleason

In [8]:
gleason_bootstraps = transform_gleason(bootstraps)

In [9]:
print(gleason_bootstraps[0])

{'embedding_mat': array([[158., 148., 145., ...,   3.,   3.,   3.],
       [151., 147., 145., ...,   2.,   2.,   2.],
       [157., 150., 147., ...,   1.,   1.,   1.],
       ...,
       [130., 130., 130., ...,   1.,   1.,   1.],
       [142., 137., 136., ...,   1.,   1.,   1.],
       [132., 130., 127., ...,   2.,   2.,   2.]]), 'images': 398     Gleason 3_1643
991      Gleason 3_978
943      Gleason 3_793
414     Gleason 3_1679
908      Gleason 3_717
             ...      
2124    Gleason 5_1716
2060    Gleason 5_1546
2671     Gleason 5_800
2204    Gleason 5_2103
2221    Gleason 5_2138
Name: img_id, Length: 2754, dtype: object, 'reduced': array([[-1.08926339e+02,  6.06829095e+01, -4.21810127e+01,
        -3.90473782e+01, -3.78418643e+01,  4.02474748e+01],
       [-2.28191828e+02,  1.56106654e+01,  6.25173226e-01,
        -9.27362598e+01,  2.31215796e+00,  6.68673887e+00],
       [ 2.12390204e+02, -2.37667765e+02, -2.66722410e+01,
        -3.37662715e+01, -1.56728087e-02, -8.34163237e